In [11]:
import requests
import mysql.connector
import re

In [12]:
db_cnx = mysql.connector.connect(user = "docker", password = "docker", host="localhost", database = "tcc")

## Buscando proposições

In [13]:
listagem_proposicoes_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
detalhe_proposicoes_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/"
##TODO: salvar isso aqui num arquivo e só consultar (ou fazer um cache)
codigo_situacao_url = "https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codSituacao"

params = {
    "ordenarPor": "siglaTipo",
    "dataInicio": "2020-01-01",
    "dataFim": "2020-12-01",
    "siglaTipo": "PL,PLC",
    "itens": "10"
}
response = requests.get(listagem_proposicoes_url, params).json()

In [14]:
proposicoes = response["dados"]

In [15]:
codigo_situacao_lista = requests.get(codigo_situacao_url).json()["dados"]
codigo_situacao = {}
for i in codigo_situacao_lista:
    codigo_situacao[i["cod"]] = i["nome"]

In [16]:
query = """
insert into proposicoes(id, tipo, ano, ementa, ementa_detalhada, uri_documento, data_apresentacao, ultimo_relator_id,
                        situacao)
    value (%(id)s, %(siglaTipo)s, %(ano)s, %(ementa)s, null, %(uri)s, null, null, null)
on duplicate key update tipo=%(siglaTipo)s, ano=%(ano)s, ementa=%(ementa)s, ementa_detalhada=null, uri_documento=%(uri)s,
                        data_apresentacao= null, ultimo_relator_id = null, situacao = null;
"""
## Campos em nulo serão atualizados mais tarde
cursor = db_cnx.cursor()
for preposicao in proposicoes:
    cursor.execute(query, preposicao)
db_cnx.commit()
cursor.close()

True

In [17]:
query = """
update proposicoes set
                       ementa_detalhada = %(ementaDetalhada)s, data_apresentacao = %(dataApresentacao)s,
                       ultimo_relator_id = %(ultimoRelatorId)s, situacao = %(situacao)s;
"""

preposicao_updates = []
cursor = db_cnx.cursor()
for preposicao in proposicoes:
    response = requests.get(f"{detalhe_proposicoes_url}/{preposicao['id']}").json()["dados"]

    params = {
        "ementaDetalhada": response["ementaDetalhada"],
        "dataApresentacao": response["dataApresentacao"]
    }

    uriUltimoRelator = response["statusProposicao"]["uriUltimoRelator"]
    ultimoRelatorId = None
    # if not uriUltimoRelator is None:
        # ultimoRelatorId = re.search(r"/deputados/(.*)$", response["statusProposicao"]["uriUltimoRelator"]).group(1)
        # TODO: preciso popular a tabela de deputados

    params["situacao"] = codigo_situacao[str(response["statusProposicao"]["codSituacao"])]
    params["ultimoRelatorId"] = ultimoRelatorId
    cursor.execute(query, params)
db_cnx.commit()
cursor.close()

True